In [95]:
import os
import ast
from tqdm import tqdm
import json
from glob import glob
from collections import Counter
import sys
sys.path.insert(0, '../')

import pandas as pd
import numpy as np

from load_data import load
from tfidf import load_tfidf
from config import Config
from utils import iterate_data_files

In [2]:
tfidf_dir = Config.tfidf_root
vocab_dir = Config.vocab_root
data_root = Config.data_root

In [9]:
START_DEV = '2018100100'
END_DEV = '2019022200'
ROOT = '../raw'

In [96]:
records = []
for path, filename in tqdm(iterate_data_files(from_dtm=START_DEV, to_dtm='2018123100', root_dir=ROOT)):
    records.append([line.strip().split() for line in open(path)])

2184it [01:29, 24.33it/s]


In [21]:
post_meta = pd.read_csv(os.path.join(ROOT, '../raw/preprocessed/post_meta_id.csv'))

In [23]:
post_meta_id = post_meta['post_meta_id'].tolist()
post_id = post_meta['post_id'].tolist()

In [52]:
post_id[0]

'@bookdb_782'

In [149]:
post_id_dict = {i:j for i, j in zip(post_id, post_meta_id)}
post_id_decode = {i:j for i, j in zip(post_meta_id, post_id)}

In [100]:
from collections import defaultdict
user_logs = defaultdict(list) # {유저식별자 : [조회글 목록]}

In [104]:
for record in tqdm(records[:5]):
    for r in record:
        user, posts_raw = r[0], r[1:]
        posts = [post_id_dict[p] for p in posts_raw if p in post_id]
        user_logs[user].extend(posts)




  0%|          | 0/5 [00:00<?, ?it/s]


 20%|██        | 1/5 [00:28<01:55, 28.85s/it]


 40%|████      | 2/5 [01:28<01:53, 37.98s/it]


 60%|██████    | 3/5 [01:53<01:08, 34.28s/it]


 80%|████████  | 4/5 [02:38<00:37, 37.55s/it]


100%|██████████| 5/5 [03:19<00:00, 39.94s/it]


In [111]:
sample_id_list = user_logs['#fb3b42b92d96f00afd5bf62d3575db53']

In [112]:
tfidf = load_tfidf(post_meta_id_list=sample_id_list, tfidf_dir=tfidf_dir, vocab_dir=vocab_dir, drop_id=True)

In [113]:
tfidf = tfidf.apply(lambda x: x / np.linalg.norm(x), axis=1)

In [114]:
user_preference = tfidf.sum(axis=0)

In [121]:
user_preference = user_preference / np.linalg.norm(user_preference)

In [122]:
user_preference

여행      0.000000
영화      0.000000
사랑      0.186879
에세이     0.000000
그림일기    0.217054
          ...   
본인      0.000000
체험학습    0.000000
선물투자    0.000000
시금치     0.000000
벌금      0.000000
Length: 7000, dtype: float64

In [135]:
post_subsample = np.random.choice(600000, 1000, replace=False).tolist() # 가중 확률도 괜찮겠다

In [137]:
tfidf_subsample = load_tfidf(post_meta_id_list=post_subsample, tfidf_dir=tfidf_dir, vocab_dir=vocab_dir, drop_id=True)

array([0.        , 0.        , 0.18687906, ..., 0.        , 0.        ,
       0.        ])

In [151]:
recommend_list = (tfidf_subsample * user_preference.values).sum(axis=1).sort_values(ascending=False).head(100).index.tolist()

In [154]:
recommend_list_decoded = [post_id_decode[i] for i in recommend_list]

In [155]:
recommend_list_decoded

['@tallguy_213',
 '@sharppen01_20',
 '@moonwindtre_240',
 '@bookfit_3642',
 '@haninorway19_103',
 '@ameliekim2016_38',
 '@chkwon91_10',
 '@mobiinside_559',
 '@leedongdong_58',
 '@psychorops_48',
 '@slist_86',
 '@drawingbutler_10',
 '@iampact_27',
 '@begini_379',
 '@ranrandambee_53',
 '@triple_26',
 '@littlechamber_16',
 '@ddaddassi_112',
 '@peregrino97_872',
 '@ubermensch_29',
 '@theedit_366',
 '@capotea_118',
 '@tomssi_55',
 '@antifreezie_2',
 '@yooym2000_162',
 '@byulkim_102',
 '@mori3_52',
 '@tiara8743_31',
 '@kateyoungheekwo_26',
 '@byulkim_106',
 '@eunmikim_146',
 '@jamon_2',
 '@hotelscomkr_45',
 '@9320km_50',
 '@csco2pw2gt_27',
 '@simplestories_63',
 '@bonfire_331',
 '@gunbbang_408',
 '@sound-travel_42',
 '@elang8151_222',
 '@simplestories_65',
 '@kohwang56_87',
 '@vhfhflya_45',
 '@glassdice_22',
 '@futurewave_492',
 '@garbar_384',
 '@lasw2324_28',
 '@eewol_41',
 '@gipyung_159',
 '@doyeonsunim_168',
 '@brunchfkx_8',
 '@marupress_149',
 '@byulkim_105',
 '@rana-na_26',
 '@kimssiban